In [2]:
import pandas as pd
import numpy as np
import re
from mechanize import Browser

Root = "C:/Users/JJvRijn/Documents/Python_Scripts"

In [58]:
browser = Browser()
browser.open("http://split4.pmfst.hr/dadp/?a=list")
response = browser.reload()
content = response.read()
names = re.findall(r'\bSP_[0-9, a-z, A-Z]*', str(content))

In [81]:
seq = []
toxic = []
toxic_type =[]
for r in names:
    URL = "http://split4.pmfst.hr/dadp/?a=kartica&id=" + str(r)
    browser = Browser()
    browser.open(URL)
    response = browser.reload()
    content = response.read()
    table = content.split(b"</table>")[1]
    results = table.split(b"</td>")
    toxic_type.append(results[5].split(b"<td>",1)[1])
    seq.append(str(results[-5]).split("b'<td>",1)[1])
    if len(re.findall(r"[-+]?\d*\.\d+|\d+", str(results[-4]))) >= 2:
        toxic.append(re.findall(r"[-+]?\d*\.\d+|\d+", str(results[-4]))[1])
    else:
        toxic.append("Nan")

In [86]:
DADP_data = pd.DataFrame()
DADP_data["ID"] = names
DADP_data["seq"] = seq
DADP_data['CONCENTRATION_µM'] = toxic
DADP_data = DADP_data[DADP_data["CONCENTRATION_µM"] != "Nan"]
DADP_data = DADP_data.reset_index(drop=True)
DADP_data["seq"] = DADP_data["seq"].str[:-1]

In [89]:
DADP_data

,ID,seq,CONCENTRATION_µM,hem_activity
0,SP_2586,LLGMIPLAISAISALSKL,40,HC50
1,SP_2587,GLWSKIKEAAKAAGKAALNAVTGLVNQGDQPS,200,HC50
2,SP_2589,GIMDTIKGAAKDVAAQLLDKLKCKITKC,200.00,HC50
3,SP_2590,FLGAIAGVAAKFLPKVFCFITKKC,8,HC50
4,SP_2591,FLPVIASVAAKVLPKVFCFITKKC,7,HC50
...,...,...,...,...
691,SP_Q1KLZ4,IIPLPLGYFAKKT,300.00,HC50
692,SP_Q2PGA7,AVDLAKIANKVLSSLF,250,HC50
693,SP_Q58T56,ILGPVIKTIGGVIGGLLKNL,90,HC50
694,SP_Q7T3K6,ALWKTMLKKLGTMALHAGKAALGAAADTISQGTQ,70.00,HC50


In [88]:
#showed me its all in micromolar for HC50
pd.Series(toxic_type).value_counts()
#so we add activity is 50% to the dataframes
DADP_data["hem_activity"] = "HC50"

In [90]:
#Because its all HC50 it all needs a concentration over 300uM
DADP_data['toxic'] = "~"
for v in range(len(DADP_data)):
    if float(DADP_data["CONCENTRATION_µM"][v]) <= 300:
        DADP_data.loc[v, "toxic"] = 1
DADP_data = DADP_data[DADP_data["toxic"]!= "~"]
DADP_data = DADP_data.reset_index(drop=True)

In [215]:
peptides = DADP_data["seq"].unique()
final_data = pd.DataFrame(peptides, columns=['seq'])
for a in range(len(DADP_data)):
    for n in range(len(peptides)):
        toxic = []
        toxic_count = 0
        if DADP_data["seq"][a] == peptides[n]:
            toxic.append(float(DADP_data["toxic"][a]))
            toxic_count = sum(toxic)/len(toxic)
            if toxic_count == len(toxic)/2:
                final_data.loc[n, "toxic"] = "~"
            if toxic_count <= len(toxic)/2:
                final_data.loc[n, "toxic"] = 0
            if toxic_count >= len(toxic)/2:
                final_data.loc[n, "toxic"] = 1

In [220]:
final_data.to_csv(
    Root + "/Programing/Data/AMP_focussed_data/DADP_data.csv",index = False, header=True)

In [1]:
final_data

NameError: name 'final_data' is not defined